In [1]:
!pip install transformers[sentencepiece]

In [2]:
from torch.nn.functional import softmax 
from transformers import MT5ForConditionalGeneration, MT5Tokenizer 

model_name = "alan-turing-institute/mt5-large-finetuned-mnli-xtreme-xnli" 
tokenizer = MT5Tokenizer.from_pretrained(model_name) 
model = MT5ForConditionalGeneration.from_pretrained(model_name)

In [3]:
sequence_to_classify = "Wen werden Sie bei der nächsten Wahl wählen? " 
candidate_labels = ["spor", "ekonomi", "politika"] 
hypothesis_template = "Dieses Beispiel ist {}."

In [4]:
ENTAILS_LABEL = "▁0" 
NEUTRAL_LABEL = "▁1" 
CONTRADICTS_LABEL = "▁2" 
label_inds = tokenizer.convert_tokens_to_ids([ 
                           ENTAILS_LABEL, 
                           NEUTRAL_LABEL, 
                           CONTRADICTS_LABEL])

In [5]:
def process_nli(premise, hypothesis): 
    return f'xnli: premise: {premise} hypothesis: {hypothesis}' 

In [6]:
pairs = [(sequence_to_classify, hypothesis_template.format(label)) for label in candidate_labels] 
seqs = [process_nli(premise=premise, 
                    hypothesis=hypothesis) 
                    for premise, hypothesis in pairs] 

In [7]:
print(seqs) 

['xnli: premise: Wen werden Sie bei der nächsten Wahl wählen?  hypothesis: Dieses Beispiel ist spor.', 'xnli: premise: Wen werden Sie bei der nächsten Wahl wählen?  hypothesis: Dieses Beispiel ist ekonomi.', 'xnli: premise: Wen werden Sie bei der nächsten Wahl wählen?  hypothesis: Dieses Beispiel ist politika.']


In [8]:
inputs = tokenizer.batch_encode_plus(seqs, return_tensors="pt", padding=True) 
out = model.generate(**inputs, output_scores=True, return_dict_in_generate=True, num_beams=1) 

In [9]:
scores = out.scores[0] 
scores = scores[:, label_inds] 

In [10]:
print(scores)

tensor([[-0.9851,  2.2550, -0.0783],
        [-5.1690, -0.7202, -2.5855],
        [ 2.7442,  3.6727,  0.7169]])


In [11]:
entailment_ind = 0 
contradiction_ind = 2 
entail_vs_contra_scores = scores[:, [entailment_ind, contradiction_ind]] 

In [12]:
entail_vs_contra_probas = softmax(entail_vs_contra_scores, dim=1) 

In [13]:
print(entail_vs_contra_probas)

tensor([[0.2877, 0.7123],
        [0.0702, 0.9298],
        [0.8836, 0.1164]])


In [14]:
entail_scores = scores[:, entailment_ind] 
entail_probas = softmax(entail_scores, dim=0) 

In [15]:
print(entail_probas) 

tensor([2.3438e-02, 3.5716e-04, 9.7620e-01])


In [16]:
print(dict(zip(candidate_labels, entail_probas.tolist()))) 

{'spor': 0.023438096046447754, 'ekonomi': 0.0003571564157027751, 'politika': 0.9762046933174133}
